In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the CSV file
file_path = "IMDb Movies India.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Normalize the case for movie details
df['Name'] = df['Name'].str.lower()
df['Genre'] = df['Genre'].str.lower()
df['Director'] = df['Director'].str.lower()
df['Actor 1'] = df['Actor 1'].str.lower()
df['Actor 2'] = df['Actor 2'].str.lower()
df['Actor 3'] = df['Actor 3'].str.lower()

# Remove parentheses from Year (if present)
df['Year'] = df['Year'].str.replace(r'\(|\)', '', regex=True)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Fill missing values
df['Genre'].fillna('unknown', inplace=True)
df['Director'].fillna('unknown', inplace=True)
df['Actor 1'].fillna('unknown', inplace=True)
df['Actor 2'].fillna('unknown', inplace=True)
df['Actor 3'].fillna('unknown', inplace=True)
df.dropna(subset=['Year', 'Rating', 'Votes'], inplace=True)

# Extract numeric value from 'Duration' and fill missing values
df['Duration'] = df['Duration'].str.extract('(\d+)').astype(float)
median_duration = df['Duration'].median()
df['Duration'].fillna(median_duration, inplace=True)

# Select features and target variable
features = ['Year', 'Duration', 'Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
X = df[features]
y = df['Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing and model pipeline
categorical_features = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
numeric_features = ['Year', 'Duration']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Create a pipeline with preprocessing and a regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

# Function to display star rating
def display_star_rating(rating):
    full_stars = int(rating)
    half_star = rating - full_stars >= 0.5
    stars = '★' * full_stars + '½' * half_star
    return stars

def get_or_predict_rating(movie_name, year, duration, genre, director, actor1, actor2, actor3):
    """
    Gets the actual rating if found, otherwise predicts the rating.

    Args:
        movie_name: The name of the movie.
        year: The release year of the movie.
        duration: The duration of the movie in minutes.
        genre: The genre of the movie.
        director: The name of the director.
        actor1: The name of the first actor.
        actor2: The name of the second actor (optional).
        actor3: The name of the third actor (optional).

    Returns:
        A string indicating the rating, whether actual or predicted.
    """

    # Normalize case for input details
    movie_name = movie_name.lower()
    genre = genre.lower()
    director = director.lower()
    actor1 = actor1.lower()
    actor2 = actor2.lower() or "unknown"
    actor3 = actor3.lower() or "unknown"

    # Exact match search
    exact_match = df[
        (df['Name'].str.contains(movie_name, case=False, na=False)) &
        (df['Year'] == year) &
        (df['Duration'] == duration) &
        (df['Genre'].str.contains(genre, case=False, na=False)) &
        (df['Director'].str.contains(director, case=False, na=False)) &
        (
            (df['Actor 1'].str.contains(actor1, case=False, na=False)) |
            (df['Actor 2'].str.contains(actor1, case=False, na=False)) |
            (df['Actor 3'].str.contains(actor1, case=False, na=False))
        ) &
        (
            (df['Actor 1'].str.contains(actor2, case=False, na=False)) |
            (df['Actor 2'].str.contains(actor2, case=False, na=False)) |
            (df['Actor 3'].str.contains(actor2, case=False, na=False)) |
            (actor2 == "unknown")
        ) &
        (
            (df['Actor 1'].str.contains(actor3, case=False, na=False)) |
            (df['Actor 2'].str.contains(actor3, case=False, na=False)) |
            (df['Actor 3'].str.contains(actor3, case=False, na=False)) |
            (actor3 == "unknown")
        )
    ]

    if not exact_match.empty:
        rating = exact_match.iloc[0]['Rating']
        votes = exact_match.iloc[0]['Votes']
        star_rating = display_star_rating(rating)
        return f'Actual Rating for "{movie_name.title()}": {rating} ({star_rating})\nVotes: {votes}'
    else:
        # Partial match search
        partial_match = df[
            (df['Director'].str.contains(director, case=False, na=False)) |
            (df['Actor 1'].str.contains(actor1, case=False, na=False)) |
            (df['Actor 2'].str.contains(actor1, case=False, na=False)) |
            (df['Actor 3'].str.contains(actor1, case=False, na=False)) |
            (df['Genre'].str.contains(genre, case=False, na=False))
        ]

        if not partial_match.empty:
            closest_match = partial_match.iloc[0]
            rating = closest_match['Rating']
            votes = closest_match['Votes']
            star_rating = display_star_rating(rating)
            return f'Closest Match Rating for "{movie_name.title()}": {rating} ({star_rating})\nVotes: {votes}'
        else:
            # Predict rating
            new_data = pd.DataFrame({
                'Year': [year],
                'Duration': [duration],
                'Genre': [genre],
                'Director': [director],
                'Actor 1': [actor1],
                'Actor 2': [actor2],
                'Actor 3': [actor3]
            })

            predicted_rating = model.predict(new_data)[0]
            star_rating = display_star_rating(predicted_rating)
            return f'Predicted Rating for "{movie_name.title()}": {predicted_rating} ({star_rating})'

# Interactive input from user
movie_name = input("Enter the movie name: ").strip()
year = int(input("Enter the year of release: ").strip())
duration = int(input("Enter the duration of the movie (in minutes): ").strip())
genre = input("Enter the genre of the movie: ").strip()
director = input("Enter the director of the movie: ").strip()
actor1 = input("Enter the name of the first actor: ").strip()
actor2 = input("Enter the name of the second actor (leave blank if none): ").strip() or "Unknown"
actor3 = input("Enter the name of the third actor (leave blank if none): ").strip() or "Unknown"

# Display the rating
print(get_or_predict_rating(movie_name, year, duration, genre, director, actor1, actor2, actor3))


Root Mean Squared Error: 1.1844666020252033
R-squared: 0.24537299394050738
Enter the movie name: Bhool Bhulaiyaa
Enter the year of release: 2007
Enter the duration of the movie (in minutes): 159
Enter the genre of the movie: Comedy, Horror, Mystery
Enter the director of the movie: Priyadarshan
Enter the name of the first actor: Akshay kumar
Enter the name of the second actor (leave blank if none): 
Enter the name of the third actor (leave blank if none): 
Actual Rating for "Bhool Bhulaiyaa": 7.3 (★★★★★★★)
Votes: 22,777


<ipython-input-34-ec134ee7fbba>:82: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  stars = '★' * full_stars + '½' * half_star


In [ ]:
1